In [4]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

In [5]:
def driver_setup():# Assigning a Headless Firefox Driver
    options = webdriver.FirefoxOptions()
    options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'  # Path to Chrome executable
    options.add_argument('E:/Programs/geckodriver.exe')
    # options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Firefox(options=options)
    return driver

def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

In [8]:
url = r'file:///C:/Users/PM/Desktop/PDF%20parsing/gmat-for-dummies-7th-edition.pdf'
driver = driver_setup() # for each link we are setting a new Firefox instance
driver.get(url)
    

The version of firefox cannot be detected. Trying with latest driver version


In [7]:
tree_elements = driver.find_elements(By.CLASS_NAME, "treeItemToggler")
question_elements = []
answer_elements = []
len(tree_elements)
for i in range(1,len(tree_elements)):
    parent_element = tree_elements[i].find_element(By.XPATH, "./..")
    sibling_elements = parent_element.find_elements(By.CLASS_NAME, "treeItem")
    for j in range(len(sibling_elements)):
        t = sibling_elements[j].find_element(By.TAG_NAME, "a").text
        if ('Sample Questions' in t):
            question_elements.append(sibling_elements[j])
            print(t)
        elif ('Answer Key' in t):
            answer_elements.append(sibling_elements[j])
            print(t)

3.4 Sample Questions
3.5 Answer Key
4.4 Sample Questions
4.5 Answer Key
5.6 Sample Questions
5.7 Answer Key


In [9]:
question_elements[2].click()
page = driver.find_element(By.ID, 'numPages').text
startpage = int(page[1:4].strip(' '))

answer_elements[2].click()
page = driver.find_element(By.ID, 'numPages').text
endpage = int(page[1:4].strip(' '))
print(startpage,endpage)
question_elements[2].click()

WebDriverException: Message: Failed to write request to stream


In [10]:
pagenavigation = driver.find_element(By.CSS_SELECTOR, 'div.splitToolbarButton.hiddenSmallView').find_elements(By.CSS_SELECTOR, 'button[title="Previous Page"], button[title="Next Page"]')
print(len(pagenavigation))

2


In [13]:
passage_list = []
question_list = []
choice_list = []
# for page in range(startpage,endpage):
for page in range(336,344):
    time.sleep(3)
    div = driver.find_element("css selector", 'div.page[data-page-number="'+str(page)+'"] span')
    span = div.find_elements("tag name", "span")

    start_saving_passage = False
    
    i=0
    while i < len(span):
        text = span[i].text.strip()
        qnopat = r'\b(?:[1-9]|[1-9][0-9]|[1-9][0-9][0-9])\.\B'
        stoppat1 = r'\b\d\.\d{1,2}\s+Sample Questions\b'
        
        if text.strip() == "Line":
            start_saving_passage = True
            start_saving_question = False
        
        elif re.search(qnopat, text):
            start_saving_passage = False
            start_saving_question = True

        if start_saving_passage:
            passage_list.append(text)

        elif start_saving_question:
            # dont forget to off choice.
            choi_pat = r'(?<!\S)\([A-M]\)(?!\S)'
            choi_pat_mch = re.findall(choi_pat, span[i].text)
            
            if choi_pat_mch:
                start_saving_choice = True
            
            elif re.search(qnopat, text):
                start_saving_choice = False
                
            elif re.search(stoppat1, text) or ( ('Guide' in text.lower()) and ('GMAT®'.lower() in text.lower()) and ('Verbal'.lower() in text.lower()) ):
                start_saving_choice = False
                start_saving_question = False
                start_saving_passage = False
                
            if start_saving_choice:
                choice_list.append(text)
            elif start_saving_question:
                question_list.append(text)
                
        elif ('Questions'.lower() in text.lower()) and ('refer to the passage'.lower() in text.lower()):
            start_saving_passage = False
        i+=1
    pagenavigation[1].click()

In [181]:
answer_elements[2].click()#----------------------------------------------ANSWERS
page = driver.find_element(By.ID, 'numPages').text
page = int(page[1:4].strip(' '))

div = driver.find_element("css selector", 'div.page[data-page-number="'+str(page)+'"] span')
span = div.find_elements("tag name", "span")

In [14]:
len(span)

0

In [183]:
i=0#----------------------------------------------ANSWERS
ans_no = []
ans = []
start_saving_answer = False
qnopat = r'\b(?:[1-9]|[1-9][0-9]|[1-9][0-9][0-9])\.\B'

while i < len(span):
    text = span[i].text.strip()
    
    if re.search(qnopat, text):
        start_saving_answer = True
        ans_no.append(text)
        i+=1
        continue
    
    if start_saving_answer:
        if text != '':
            ans.append(text)
            
    i+=1

In [184]:
ans_data = {'ans_number': ans_no, 'ans': ans} #----------------------------------------------ANSWERS
ans_df = pd.DataFrame(ans_data)
ans_df['ans_number'] = ans_df['ans_number'].str.replace('.','')

C:\Users\PM\AppData\Local\Temp\ipykernel_1364\768121153.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ans_df['ans_number'] = ans_df['ans_number'].str.replace('.','')


In [15]:
print(passage_list,'\n',question_list,'\n',choice_list)
print(len(passage_list),len(question_list),len(choice_list))

[] 
 [] 
 []
0 0 0


In [169]:
qnopat = r'\b(?:[1-9]|[1-9][0-9]|[1-9][0-9][0-9])\.\B'

# Initialize variables to store question numbers and content
question_numbers = []
question_content = []

startconcat = False

question = []
for item in question_list:
    match = re.match(qnopat, item)
    if match:
        if startconcat:
            question_content.append(' '.join(question))
        question = []
        question_numbers.append(item)
        startconcat = True
        continue
    if startconcat:
        question.append(item)

if question and startconcat:
    question_content.append(' '.join(question))

#------------------------------------------------------------------------------------------------------
choiinpat = r'(?<!\S)\([A-M]\)(?!\S)'

# Initialize variables to store choices
choice_numbers = []
choice_subchoices = []

current_choice_number = None
current_subchoices = []

for item in choice_list:
    cmatch = re.match(choiinpat, item)
    if cmatch:
        if current_choice_number is not None:
            choice_numbers.append(current_choice_number)
            choice_subchoices.append(current_subchoices)
            current_subchoices = []
        
        current_choice_number = item
        continue
    
    if current_choice_number is not None:
        current_subchoices.append(item)

# Append the last set of choices
if current_choice_number is not None:
    choice_numbers.append(current_choice_number)
    choice_subchoices.append(current_subchoices)
    
choice_data = []
current_values = []

one_r = {}
for i in range(len(choice_numbers)):
    if choice_numbers[i] == '(A)':
        if one_r != {}:
            choice_data.append(one_r)
        one_r = {}
    one_r['choice_'+choice_numbers[i]] = ' '.join(choice_subchoices[i])

# Append the last one_r if it's not empty
if one_r:
    choice_data.append(one_r)
#------------------------------------------------------------------------------------------------------

In [11]:
# question_content.append('pick_from_left')

In [170]:
for i in range(len(question_numbers)):
    print(question_numbers[i],'------',question_content[i])

3. ------  However much United States voters may agree that there is waste in government and that the government as a whole spends beyond its means, it is difﬁcult to ﬁnd broad support for a movement toward a minimal state.
4. ------  The voluminous personal papers of Thomas Alva Edison reveal that his inventions typically sprang to life not in a ﬂash of inspiration but evolved slowly from previous works.
1. ------  Like ants, termites have an elaborate social structure in which a few individuals reproduce and the rest are serving the colony by tending juveniles, gathering food, building the nest, or they battle intruders.
2. ------  Some bat caves, like honeybee hives, have residents that take on different duties such as defending the entrance, acting as sentinels and to sound a warning at the approach of danger, and scouting outside the cave for new food and roosting sites.
5. ------  Hundreds of species of ﬁsh generate and discharge electric currents, in bursts or as steady electric

In [171]:
choice_df = pd.DataFrame(choice_data)
question_data = {'Question Number': question_numbers, 'Question Content': question_content}

df = pd.concat([pd.DataFrame(question_data), choice_df], axis=1)
# tdf = df.copy()

In [180]:
df

,Question Number,Question Content,choice_(A),choice_(B),choice_(C),choice_(D),choice_(E)
0,1,"Like ants, termites have an elaborate social ...","are serving the colony by tending juveniles, ...",are serving the colony in that they tend juve...,"serve the colony, tending juveniles, gatherin...","serve the colony by tending juveniles, gather...","serve the colony by tending juveniles, gather..."
1,2,"Some bat caves, like honeybee hives, have res...",acting as sentinels and to sound,acting as sentinels and sounding,to act as sentinels and sound,to act as sentinels and to sound,to act as a sentinel sounding
2,3,However much United States voters may agree t...,However much United States voters may agree that,Despite the agreement among United States vot...,Although United States voters agree,Even though United States voters may agree,There is agreement among United States voters...
3,4,The voluminous personal papers of Thomas Alva...,sprang to life not in a ﬂash of inspiration b...,sprang to life not in a ﬂash of inspiration b...,did not spring to life in a ﬂash of inspirati...,did not spring to life in a ﬂash of inspirati...,did not spring to life in a ﬂash of inspirati...
4,5,Hundreds of species of ﬁsh generate and disch...,"either to ﬁnd and attack prey, to defend them...","either for ﬁnding and attacking prey, defend ...","to ﬁnd and attack prey, for defense, or commu...","for ﬁnding and attacking prey, to defend them...","to ﬁnd and attack prey, to defend themselves,..."
...,...,...,...,...,...,...,...
108,109,The nephew of Pliny the Elder wrote the only ...,The nephew of Pliny the Elder wrote the only ...,"To the historian Tacitus, the nephew of Pliny...",The only eyewitness account is in two letters...,"Writing the only eyewitness account, Pliny th...","In two letters to the historian Tacitus, the ..."
109,110,Being a United States citizen since 1988 and ...,Being a United States citizen since 1988 and ...,Having been a United States citizen since 198...,"Born in Calcutta in 1940, author Bharati Mukh...",Being born in Calcutta in 1940 and having bee...,Having been born in Calcutta in 1940 and bein...
110,111,Initiated ﬁve centuries after Europeans arriv...,Initiated ﬁve centuries after Europeans arriv...,"Initiated on Columbus Day 1992, ﬁve centuries...","Initiated on Columbus Day 1992, ﬁve centuries...",Pledging a $100 million investment in the sea...,Pledging a $100 million investment in the sea...
111,112,In the 1980s the federal government was the l...,In the 1980s the federal government was the l...,The federal government was the largest single...,In the 1980s the federal government was the l...,The largest single provider of day care for c...,In the 1980s the largest single provider of d...


In [14]:
passage_numbers = []#-----------------------------------------------------------passage
passage_content = []

startconcat = False

passage = []
for item in passage_list:
    if item.lower() == 'Line'.lower():
        if startconcat:
            passage_content.append(' '.join(passage))
        passage = []
        passage_numbers.append(item)
        startconcat = True
        continue
    if startconcat:
        passage.append(item)

if passage and startconcat:
    passage_content.append(' '.join(passage))

In [46]:
for i in range(len(passage_content)):#-----------------------------------------------------------passage
    print(i+1,passage_content[i][-50:])

1 ubtract. Questions 1–6 refer to the passage above.
2 waters. Questions 7–11 refer to the passage above.
3 emination of another mosquito,  Aedes albopictus .
4 esses. Questions 18–23 refer to the passage above.
5  salt. Questions 24–26 refer to the passage above.
6 eered. Questions 27–32 refer to the passage above.
7 lture. Questions 33–38 refer to the passage above.
8 mmick. Questions 39–44 refer to the passage above.
9 e war. Questions 45–49 refer to the passage above.
10 omers. Questions 50–55 refer to the passage above.
11 ating. Questions 56–60 refer to the passage above.
12 grove. Questions 61–63 refer to the passage above.
13 rs do. Questions 64–70 refer to the passage above.
14 e of the most effective compounds in both regards.
15  concentrations and stimulating it at higher ones.
16 ale” jobs that women had been permitted to master.
17 d as irrelevant the family roles important to many
18 merge. Questions 85–90 refer to the passage above.
19 opulation of galaxies that are dif

In [40]:
pasdf = pd.DataFrame(passage_content, columns=['passage_content'])#-----------------------------------------------------------passage
pasdf

,passage_content
0,(5) (10) (15) (20) (25) The Gross Domestic Pro...
1,(5) (10) (15) (20) (25) Coral reefs are one of...
2,(5) (10) (15) (20) (25) (30) Although genetic ...
3,"(5) (10) (15) (20) (25) (30) In 1994, a team o..."
4,(5) (10) (15) (20) (25) (30) (35) (40) (45) Ho...
5,(5) (10) (15) (20) (25) (30) The new school of...
6,(5) (10) (15) (20) (25) (30) (35) (40) (45) At...
7,(5) (10) (15) (20) (25) (30) Seeking a competi...
8,(5) (10) (15) (20) (25) (30) In a 1918 editori...
9,(5) (10) (15) (20) (25) The fact that superior...


In [48]:
pattern = r'Questions \d+–\d+ refer to the passage above'#-----------------------------------------------------------passage

# Function to extract pattern text
def extract_pattern(text):
    match = re.search(pattern, text)
    if match:
        return match.group()
    else:
        return None
pasdf['qno'] = pasdf['passage_content'].apply(extract_pattern)

In [51]:
def extract_numbers(pattern_text):#-----------------------------------------------------------passage
    if pattern_text:
        return re.search(r'\d+–\d+', pattern_text).group()
    else:
        return None

# Apply the function to create a new column 'qno_numbers'
pasdf['qno'] = pasdf['qno'].apply(extract_numbers)

In [70]:
# pasdf['start'] = pasdf['qno'].apply(lambda x: (x.split('–')[0] if x else x)) 
# pasdf['end'] = pasdf['qno'].apply(lambda x: (x.split('–')[1] if x else x)) #-----------------------------------------------------------passage
# del pasdf['qno']
pasdf.to_csv('pasdf.csv',index = False)

In [78]:
pasdf = pd.read_csv('pasdf.csv')#-----------------------------------------------------------passage

In [79]:
pasdf = pasdf.groupby(['start', 'end'], as_index=False)['passage_content'].apply(' '.join)#-----------------------------------------------------------passage
pasdf = pasdf.reset_index().rename(columns={0: 'merged_passage_content'})
del pasdf['index']

In [88]:
pasdf

,start,end,passage_content
0,1,6,(5) (10) (15) (20) (25) The Gross Domestic Pro...
1,7,11,(5) (10) (15) (20) (25) Coral reefs are one of...
2,12,17,(5) (10) (15) (20) (25) (30) Although genetic ...
3,18,23,"(5) (10) (15) (20) (25) (30) In 1994, a team o..."
4,24,26,(5) (10) (15) (20) (25) (30) (35) (40) (45) Ho...
5,27,32,(5) (10) (15) (20) (25) (30) The new school of...
6,33,38,(5) (10) (15) (20) (25) (30) (35) (40) (45) At...
7,39,44,(5) (10) (15) (20) (25) (30) Seeking a competi...
8,45,49,(5) (10) (15) (20) (25) (30) In a 1918 editori...
9,50,55,(5) (10) (15) (20) (25) The fact that superior...


In [81]:
pasdf.to_csv('pasdf.csv',index = False)

In [173]:
for index, row in df.iterrows():#-----------------------------------------------------------question
    question_number = str(row['Question Number'])
    if not question_number[:-1].isdigit():
        # Extract the text from the next column
        nearby_text = row['Question Content']
        if isinstance(nearby_text, str):
            # Append the text to the nearby question content
            updated_content = question_number+' '+nearby_text
            df.at[index, 'Question Content'] = updated_content
df

,Question Number,Question Content,choice_(A),choice_(B),choice_(C),choice_(D),choice_(E)
0,3.,However much United States voters may agree t...,However much United States voters may agree that,Despite the agreement among United States vot...,Although United States voters agree,Even though United States voters may agree,There is agreement among United States voters...
1,4.,The voluminous personal papers of Thomas Alva...,sprang to life not in a ﬂash of inspiration b...,sprang to life not in a ﬂash of inspiration b...,did not spring to life in a ﬂash of inspirati...,did not spring to life in a ﬂash of inspirati...,did not spring to life in a ﬂash of inspirati...
2,1.,"Like ants, termites have an elaborate social ...","are serving the colony by tending juveniles, ...",are serving the colony in that they tend juve...,"serve the colony, tending juveniles, gatherin...","serve the colony by tending juveniles, gather...","serve the colony by tending juveniles, gather..."
3,2.,"Some bat caves, like honeybee hives, have res...",acting as sentinels and to sound,acting as sentinels and sounding,to act as sentinels and sound,to act as sentinels and to sound,to act as a sentinel sounding
4,5.,Hundreds of species of ﬁsh generate and disch...,"either to ﬁnd and attack prey, to defend them...","either for ﬁnding and attacking prey, defend ...","to ﬁnd and attack prey, for defense, or commu...","for ﬁnding and attacking prey, to defend them...","to ﬁnd and attack prey, to defend themselves,..."
...,...,...,...,...,...,...,...
108,107.,"Bluegrass musician Bill Monroe, whose reperto...",were inﬂuential on generations of bluegrass a...,"inﬂuenced generations of bluegrass artists, a...",was inﬂuential to generations of bluegrass ar...,was inﬂuential to generations of bluegrass ar...,were an inﬂuence on generations of bluegrass ...
109,108. Sound can travel through water for enormous,108. Sound can travel through water for enormo...,prevented from dissipating its acoustic energ...,prevented from having its acoustic energy dis...,its acoustic energy prevented from dissipatin...,its acoustic energy prevented from being diss...,preventing its acoustic energy from dissipati...
110,112.,In the 1980s the federal government was the l...,In the 1980s the federal government was the l...,The federal government was the largest single...,In the 1980s the federal government was the l...,The largest single provider of day care for c...,In the 1980s the largest single provider of d...
111,113.,"In A.D. 391, resulting from the destruction o...",resulting from the destruction of the largest...,the destroying of the largest library of the ...,because of the result of the destruction of t...,as a result of the destruction of the library...,Alexandria’s largest library of the ancient w...


In [174]:
df['Question Number'] = df['Question Number'].apply(lambda x: (x[:3] if len(x) > 4 else x))#-----------------------------------------------------------question
df['Question Content'] = df.apply(lambda row: re.sub(r'^\d+\.\s*', '', row['Question Content']) if pd.notnull(row['Question Content']) else '', axis=1)
# correct numbers to have numbers and questions to have question

In [175]:
df['Question Number'] = df['Question Number'].str.replace('.','')

C:\Users\PM\AppData\Local\Temp\ipykernel_1364\3995893735.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Question Number'] = df['Question Number'].str.replace('.','')


In [176]:
print(len(question_content),len(question_numbers))

113 113


In [177]:
df['Question Number'] = df['Question Number'].astype('int')


In [178]:
df = df.sort_values(by='Question Number')
df = df.reset_index(drop=True)

In [30]:
# df['Question Content'] = df['Question Content'].str.replace('pick_from_left','')

In [185]:
ans_df['ans_number'] = ans_df['ans_number'].astype('int')#----------------------------------------------ANSWERS

In [95]:
df = df.merge(pasdf, how='left', left_on=['Question Number'], right_on=['start'])#-----------------------------------------------------------passage
df.drop(['start', 'end'], axis=1, inplace=True)
df['passage_content'] = df['passage_content'].fillna(method='ffill')
df

,Question Number,Question Content,choice_(A),choice_(B),choice_(C),choice_(D),choice_(E),passage_content
0,1,The primary purpose of the passage is to,identify ways in which the GDP could be modiﬁ...,"suggest that the GDP, in spite of certain sho...",examine crucial shortcomings of the GDP as an...,argue that the growth of the United States ec...,discuss how the GDP came to be used as the pr...,(5) (10) (15) (20) (25) The Gross Domestic Pro...
1,2,Which of the following best describes the fun...,It describes an assumption about the GDP that...,It contributes to a discussion of the origins...,It clariﬁes a common misconception about the ...,It identiﬁes a major ﬂaw in the GDP.,It suggests a revision to the method of calcu...,(5) (10) (15) (20) (25) The Gross Domestic Pro...
2,3,It can be inferred that the author of the pas...,It is a comprehensive indicator of a nation’s...,It is not accurately captured by the price of...,It is usually less than the intrinsic value o...,It is more difﬁcult to calculate than the eco...,It is calculated differently in capitalist co...,(5) (10) (15) (20) (25) The Gross Domestic Pro...
3,4,The comparison of the GDP to a calculating ma...,Refute an assertion that the calculations inv...,Indicate that the GDP is better suited to rec...,Suggest that it is likely that the GDP will b...,Illustrate the point that the GDP has no way ...,Exemplify an assertion that the GDP tends to ...,(5) (10) (15) (20) (25) The Gross Domestic Pro...
4,5,The passage implies that national policies th...,become increasingly capitalistic in nature,disregard the economic importance of environm...,overestimate the amount of commercial activit...,overestimate the economic signiﬁcance of cohe...,assume that the economic signiﬁcance of goods...,(5) (10) (15) (20) (25) The Gross Domestic Pro...
...,...,...,...,...,...,...,...,...
99,100,It can be inferred from the passage that the a...,an inevitable problem in a weak national economy,a positive sign of employee concern about a c...,a predictor of employee reactions to a compan...,a phenomenon found more often in state-owned ...,a deterrence to high performance levels in an...,(50) miss Thomas Paine’s point that “what we o...
100,101,The passage supports which of the following s...,"At three different companies, approximately n...",Approximately 90 percent of the eligible work...,The opportunity to buy shares was discouraged...,Companies that demonstrated the highest produ...,Eligibility to buy shares was contingent on e...,(50) miss Thomas Paine’s point that “what we o...
101,102,Which of the following statements is most con...,A democratic government that decides it is in...,The ideal way for a government to protect emp...,The failure to harness the power of self-inte...,Governments that want to implement privatizat...,The individual shareholder will reap only a m...,(50) miss Thomas Paine’s point that “what we o...
102,103,Which of the following can be inferred from t...,It depends to a potentially dangerous degree ...,It conforms in its most general outlines to T...,It was originally conceived to include some g...,"It has been successful, even though privatiza...",It is taking place more slowly than some econ...,(50) miss Thomas Paine’s point that “what we o...


In [186]:
df = df.merge(ans_df, how='left', left_on=['Question Number'], right_on=['ans_number'])#----------------------------------------------ANSWERS
del df['ans_number']
df

,Question Number,Question Content,choice_(A),choice_(B),choice_(C),choice_(D),choice_(E),ans
0,1,"Like ants, termites have an elaborate social ...","are serving the colony by tending juveniles, ...",are serving the colony in that they tend juve...,"serve the colony, tending juveniles, gatherin...","serve the colony by tending juveniles, gather...","serve the colony by tending juveniles, gather...",E
1,2,"Some bat caves, like honeybee hives, have res...",acting as sentinels and to sound,acting as sentinels and sounding,to act as sentinels and sound,to act as sentinels and to sound,to act as a sentinel sounding,B
2,3,However much United States voters may agree t...,However much United States voters may agree that,Despite the agreement among United States vot...,Although United States voters agree,Even though United States voters may agree,There is agreement among United States voters...,A
3,4,The voluminous personal papers of Thomas Alva...,sprang to life not in a ﬂash of inspiration b...,sprang to life not in a ﬂash of inspiration b...,did not spring to life in a ﬂash of inspirati...,did not spring to life in a ﬂash of inspirati...,did not spring to life in a ﬂash of inspirati...,C
4,5,Hundreds of species of ﬁsh generate and disch...,"either to ﬁnd and attack prey, to defend them...","either for ﬁnding and attacking prey, defend ...","to ﬁnd and attack prey, for defense, or commu...","for ﬁnding and attacking prey, to defend them...","to ﬁnd and attack prey, to defend themselves,...",E
...,...,...,...,...,...,...,...,...
108,109,The nephew of Pliny the Elder wrote the only ...,The nephew of Pliny the Elder wrote the only ...,"To the historian Tacitus, the nephew of Pliny...",The only eyewitness account is in two letters...,"Writing the only eyewitness account, Pliny th...","In two letters to the historian Tacitus, the ...",E
109,110,Being a United States citizen since 1988 and ...,Being a United States citizen since 1988 and ...,Having been a United States citizen since 198...,"Born in Calcutta in 1940, author Bharati Mukh...",Being born in Calcutta in 1940 and having bee...,Having been born in Calcutta in 1940 and bein...,C
110,111,Initiated ﬁve centuries after Europeans arriv...,Initiated ﬁve centuries after Europeans arriv...,"Initiated on Columbus Day 1992, ﬁve centuries...","Initiated on Columbus Day 1992, ﬁve centuries...",Pledging a $100 million investment in the sea...,Pledging a $100 million investment in the sea...,C
111,112,In the 1980s the federal government was the l...,In the 1980s the federal government was the l...,The federal government was the largest single...,In the 1980s the federal government was the l...,The largest single provider of day care for c...,In the 1980s the largest single provider of d...,E


In [190]:
df.columns

Index(['Question Number', 'Question Content', 'choice_(A)', 'choice_(B)',
       'choice_(C)', 'choice_(D)', 'choice_(E)', 'ans'],
      dtype='object')

In [188]:
# cust = ['passage_content','Question Number', 'Question Content', 'choice_(A)', 'choice_(B)', 'choice_(C)', 'choice_(D)', 'choice_(E)', 'ans']
# cust = ['Question Number', 'Question Content', 'choice_(A)', 'choice_(B)', 'choice_(C)', 'choice_(D)', 'choice_(E)', 'ans']
cust = ['Question Number', 'Question Content', 'choice_(A)', 'choice_(B)', 'choice_(C)', 'choice_(D)', 'choice_(E)']
df = df.reindex(columns = cust)

In [189]:
# df['passage']= ' '.join(passage_list)
df.to_excel('Gmat_5.4_Sample_Questions.xlsx', index = False)
